In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import torch 

from torch import nn

In [2]:
class NeuralNetwork(nn.Module):

    activation_fn: callable

    def __init__(self, arq, activation_fn):

        super(NeuralNetwork, self).__init__()

        self.activation_fn = activation_fn

        self.neural_network = nn.ModuleList()
        
        for i in range(len(arq)-1):
            self.neural_network.append(nn.Linear(arq[i], arq[i+1]))
            nn.init.xavier_normal_(self.neural_network[i].weight)

    
    def forward(self, x):

        for layer in self.neural_network[:-1]: x = self.activation_fn(layer(x))
        x = self.neural_network[-1](x)  # Sem ativação na última camada
        return x
    
    def save(self, local: str):

        torch.save(self.state_dict(), local)

    def load(self, local: str):

        self.load_state_dict(torch.load(local))

In [7]:
class Inversive_PINN:

    def __init__(self):

        self.param = nn.Parameter(torch.tensor(1., requires_grad = True))

    def residual_loss(self, model, x):

        # Passagem da entrada pela rede
        x.requires_grad_(True)
        u = model(x)

        # Calculo dos gradientes
        du = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        d2u = torch.autograd.grad(du, x, grad_outputs=torch.ones_like(du), create_graph=True)[0]
        
        # Resíduo da equação diferencial
        residual = d2u + self.param * u
        
        return torch.mean(residual**2)
    
    def data_loss(self, model, input, target):

        u_pred = model(input)
        return torch.mean((u_pred - target)**2)
    
    def trainning(self, model, x_data, u_data, x_residual, epochs = 1_000, learning_rate = 1e-3, step = 100):

        optmizer = torch.optim.Adam([
            {'params': model.parameters()},
            {'params': [self.param]}
            ],
            lr = learning_rate
        )
        loss_history = []

        for ep in range(epochs):

            optmizer.zero_grad()

            loss_data = self.data_loss(model, x_data, u_data)
            loss_residual = self.residual_loss(model, x_residual)

            loss = loss_data + loss_residual
            loss.backward()

            optmizer.step()

            if ep % step == 0 : print(f'Epoch {ep}, Loss: {loss.item()}, Lambda: {self.param.item()}')
            loss_history.append(loss.item())

        return loss_history


In [8]:
def generate_data(lambda_true=5.0, n_data=100, noise_level=0.05):

    x_data = torch.linspace(0, 1, n_data).view(-1, 1)
    u_data = torch.sin(np.sqrt(lambda_true) * x_data)
    u_data += noise_level * torch.randn_like(u_data)
    
    return x_data, u_data

In [13]:
model = NeuralNetwork(
    [1] + 3 * [100] + [1],
    nn.Tanh()
)
inverse_pinn = Inversive_PINN()

x_data, u_data = generate_data(lambda_true = 4.0, n_data = 500)
x_residual = torch.linspace(0, 1, 100).view(-1, 1).requires_grad_(True)

x_data.shape, u_data.shape

(torch.Size([500, 1]), torch.Size([500, 1]))

In [14]:
loss_history = inverse_pinn.trainning(model, x_data, u_data, x_residual, epochs = 50_000, step = 1_000)

Epoch 0, Loss: 1.169370412826538, Lambda: 0.9990000128746033
Epoch 1000, Loss: 0.010887625627219677, Lambda: 0.996890664100647
Epoch 2000, Loss: 0.010426095686852932, Lambda: 1.0880730152130127
Epoch 3000, Loss: 0.009766711853444576, Lambda: 1.2216764688491821
Epoch 4000, Loss: 0.009012575261294842, Lambda: 1.3834940195083618
Epoch 5000, Loss: 0.008179369382560253, Lambda: 1.5646663904190063
Epoch 6000, Loss: 0.007425185292959213, Lambda: 1.7422140836715698
Epoch 7000, Loss: 0.0067487978376448154, Lambda: 1.9126628637313843
Epoch 8000, Loss: 0.00622345507144928, Lambda: 2.07961368560791
Epoch 9000, Loss: 0.005586522631347179, Lambda: 2.2393221855163574
Epoch 10000, Loss: 0.005135396495461464, Lambda: 2.383657217025757
Epoch 11000, Loss: 0.0047171940095722675, Lambda: 2.5164287090301514
Epoch 12000, Loss: 0.004397244658321142, Lambda: 2.6447978019714355
Epoch 13000, Loss: 0.004200169816613197, Lambda: 2.76465106010437
Epoch 14000, Loss: 0.003813691670075059, Lambda: 2.869314670562744
Ep